In [3]:
import requests
from bs4 import BeautifulSoup

In [4]:
url = "https://shopee.co.id/search?keyword=cabai%20rawit"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

In [5]:
response

<Response [200]>

# Shopee

### Manual

In [ ]:
import undetected_chromedriver as uc
import time

options = uc.ChromeOptions()
options.add_argument("--window-size=1200,800")

driver = uc.Chrome(options=options)

driver.get("https://shopee.co.id/buyer/login")
print("🔐 Silakan login secara manual ke Shopee...")
time.sleep(45)  

driver.get("https://shopee.co.id/search?keyword=cabai%20merah")
time.sleep(5)

🔐 Silakan login secara manual ke Shopee...


In [ ]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def slow_scroll(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")
    for y in range(0, last_height, 400):  
        driver.execute_script(f"window.scrollTo(0, {y});")
        time.sleep(0.3)
    time.sleep(2.5)  

products = []
page = 1
max_page = 10  

while page <= max_page:
    print(f"\n📄 Sedang scraping halaman {page}...")

    WebDriverWait(driver, 15).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "shopee-search-item-result__item"))
    )

    slow_scroll(driver) 

    soup = BeautifulSoup(driver.page_source, "html.parser")
    product_cards = soup.select(".shopee-search-item-result__item")
    print(f"🔎 Ditemukan {len(product_cards)} produk (termasuk iklan)")

    parsed_count = 0
    skipped_count = 0

    for card in product_cards:
        try:
            name_tag = card.select_one("div.line-clamp-2")
            price_tag = card.select_one("span.font-medium.text-base\\/5.truncate")
            
            if not name_tag or not price_tag:
                skipped_count += 1
                continue  

            name = name_tag.text.strip()
            price = price_tag.text.strip()

            sold_tag = card.select_one("div.truncate.text-shopee-black87.text-xs.min-h-4")
            sold = sold_tag.text.strip() if sold_tag else "Tidak ditemukan"

            location_tag = card.select_one("div.flex-shrink.min-w-0.truncate.text-shopee-black54")
            location = location_tag.text.strip() if location_tag else "Tidak ditemukan"

            rating_tag = card.select_one("div.text-shopee-black87.text-xs\\/sp14.flex-none")
            rating = rating_tag.text.strip() if rating_tag else "Tidak ditemukan"

            products.append({
                "product_name": name,
                "price": price,
                "sold": sold,
                "location": location,
                "rating": rating
            })
            parsed_count += 1

        except Exception as e:
            skipped_count += 1
            continue

    print(f"✅ Produk berhasil diparsing: {parsed_count}, dilewati: {skipped_count}")

    # Pindah ke halaman berikutnya
    page += 1
    if page > max_page:
        break

    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, f'//a[@class="shopee-button-no-outline" and text()="{page}"]'))
        )
        next_button.click()
        time.sleep(4) 
    except:
        print("🚫 Gagal klik ke halaman berikutnya.")
        break

df = pd.DataFrame(products)
df.to_csv("shopee_cabai_rawit.csv", index=False)
print(f"\n📦 Data berhasil disimpan ke 'shopee_cabai_rawit.csv' ({len(products)} produk)")

driver.quit()

### Refactor

In [ ]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc

def slow_scroll(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")
    for y in range(0, last_height, 400):
        driver.execute_script(f"window.scrollTo(0, {y});")
        time.sleep(0.3)
    time.sleep(2.5)

def scrape_shopee(keyword: str, max_page: int = 5, output_file: str = "hasil_scraping.csv"):
    options = uc.ChromeOptions()
    options.add_argument("--window-size=1200,800")
    driver = uc.Chrome(options=options)

    driver.get("https://shopee.co.id/buyer/login")
    print("🔐 Silakan login manual ke Shopee...")
    time.sleep(45) 
    
    driver.get(f"https://shopee.co.id/search?keyword={keyword.replace(' ', '%20')}")
    time.sleep(10)

    products = []
    page = 1

    while page <= max_page:
        print(f"\n📄 Scraping halaman {page}...")

        try:
            WebDriverWait(driver, 15).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, "shopee-search-item-result__item"))
            )
        except:
            print("❗ Produk tidak muncul, halaman mungkin gagal dimuat.")
            break

        slow_scroll(driver)

        soup = BeautifulSoup(driver.page_source, "html.parser")
        product_cards = soup.select(".shopee-search-item-result__item")
        print(f"🔎 {len(product_cards)} produk ditemukan")

        for card in product_cards:
            try:
                name_tag = card.select_one("div.line-clamp-2")
                price_tag = card.select_one("span.font-medium.text-base\\/5.truncate")
                if not name_tag or not price_tag:
                    continue

                name = name_tag.text.strip()
                price = price_tag.text.strip()

                sold_tag = card.select_one("div.truncate.text-shopee-black87.text-xs.min-h-4")
                sold = sold_tag.text.strip() if sold_tag else "Tidak ditemukan"

                location_tag = card.select_one("div.flex-shrink.min-w-0.truncate.text-shopee-black54")
                location = location_tag.text.strip() if location_tag else "Tidak ditemukan"

                rating_tag = card.select_one("div.text-shopee-black87.text-xs\\/sp14.flex-none")
                rating = rating_tag.text.strip() if rating_tag else "Tidak ditemukan"

                products.append({
                    "product_name": name,
                    "price": price,
                    "sold": sold,
                    "location": location,
                    "rating": rating,
                    "kategori": keyword
                })

            except Exception:
                continue

        page += 1
        if page > max_page:
            break

        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, f'//a[@class="shopee-button-no-outline" and text()="{page}"]'))
            )
            next_button.click()
            time.sleep(4)
        except:
            print("🚫 Gagal klik halaman berikutnya.")
            break

    # Simpan ke file
    df = pd.DataFrame(products)
    df.to_csv(output_file, index=False)
    print(f"\n✅ {len(products)} produk disimpan ke '{output_file}'")

    driver.quit()

In [19]:
scrape_shopee("iphone 11", max_page=5, output_file="iphone_11.csv")

🔐 Silakan login manual ke Shopee...

📄 Scraping halaman 1...
🔎 60 produk ditemukan

📄 Scraping halaman 2...
🔎 60 produk ditemukan

📄 Scraping halaman 3...
🔎 60 produk ditemukan

📄 Scraping halaman 4...
🔎 60 produk ditemukan

📄 Scraping halaman 5...
🔎 60 produk ditemukan

✅ 300 produk disimpan ke 'iphone_11.csv'


### Scrapping Dengan GUI

In [ ]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc
import tkinter as tk
from tkinter import ttk, messagebox

def slow_scroll(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")
    for y in range(0, last_height, 400):
        driver.execute_script(f"window.scrollTo(0, {y});")
        time.sleep(0.3)
    time.sleep(2.5)

def wait_for_login(driver, timeout=180):
    try:
        WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.CLASS_NAME, "shopee-searchbar-input__input"))
        )
        print("✅ Login selesai.")
    except:
        raise Exception("⏳ Login gagal atau belum selesai.")

def wait_for_captcha(driver, timeout=180):
    try:
        WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.CLASS_NAME, "shopee-searchbar-input__input"))
        )
        print("✅ CAPTCHA selesai.")
    except:
        raise Exception("⏳ CAPTCHA belum selesai atau gagal.")

def scrape_shopee(keyword: str, max_page: int, output_file: str):
    try:
        options = uc.ChromeOptions()
        options.add_argument("--window-size=1200,800")
        driver = uc.Chrome(options=options)

        driver.get("https://shopee.co.id/buyer/login")
        print("🔐 Silakan login ke Shopee.")
        messagebox.showinfo("Login", "Silakan login manual ke Shopee di browser.\nKlik OK jika sudah login.")
        wait_for_login(driver)

        messagebox.showinfo("CAPTCHA", "Jika muncul CAPTCHA, silakan selesaikan dulu.\nKlik OK jika sudah selesai CAPTCHA.")
        wait_for_captcha(driver)

        driver.get(f"https://shopee.co.id/search?keyword={keyword.replace(' ', '%20')}")
        time.sleep(5)

        products = []
        page = 1

        while page <= max_page:
            print(f"\n📄 Scraping halaman {page}...")

            try:
                WebDriverWait(driver, 15).until(
                    EC.presence_of_all_elements_located((By.CLASS_NAME, "shopee-search-item-result__item"))
                )
            except:
                print("❗ Produk tidak muncul, halaman mungkin gagal dimuat.")
                break

            slow_scroll(driver)
            soup = BeautifulSoup(driver.page_source, "html.parser")
            product_cards = soup.select(".shopee-search-item-result__item")
            print(f"🔎 {len(product_cards)} produk ditemukan")

            for card in product_cards:
                try:
                    name_tag = card.select_one("div.line-clamp-2")
                    price_tag = card.select_one("span.font-medium.text-base\\/5.truncate")
                    if not name_tag or not price_tag:
                        continue

                    name = name_tag.text.strip()
                    price = price_tag.text.strip()

                    sold_tag = card.select_one("div.truncate.text-shopee-black87.text-xs.min-h-4")
                    sold = sold_tag.text.strip() if sold_tag else "Tidak ditemukan"

                    location_tag = card.select_one("div.flex-shrink.min-w-0.truncate.text-shopee-black54")
                    location = location_tag.text.strip() if location_tag else "Tidak ditemukan"

                    rating_tag = card.select_one("div.text-shopee-black87.text-xs\\/sp14.flex-none")
                    rating = rating_tag.text.strip() if rating_tag else "Tidak ditemukan"

                    products.append({
                        "product_name": name,
                        "price": price,
                        "sold": sold,
                        "location": location,
                        "rating": rating,
                        "kategori": keyword
                    })
                except:
                    continue

            page += 1
            if page > max_page:
                break

            try:
                next_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, f'//a[@class="shopee-button-no-outline" and text()="{page}"]'))
                )
                next_button.click()
                time.sleep(4)
            except:
                print("🚫 Gagal klik halaman berikutnya.")
                break

        df = pd.DataFrame(products)
        df.to_csv(output_file, index=False)
        print(f"\n✅ {len(products)} produk disimpan ke '{output_file}'")
        messagebox.showinfo("Selesai", f"Scraping selesai!\n{len(products)} produk disimpan ke '{output_file}'")
    except Exception as e:
        messagebox.showerror("Error", str(e))
    finally:
        driver.quit()

# ========== GUI ==========
def start_scraping():
    keyword = keyword_entry.get()
    pages = pages_entry.get()
    filename = filename_entry.get()

    if not keyword or not pages or not filename:
        messagebox.showwarning("Input Kosong", "Semua kolom harus diisi.")
        return

    try:
        pages = int(pages)
    except ValueError:
        messagebox.showerror("Input Salah", "Jumlah halaman harus berupa angka.")
        return

    if not filename.endswith(".csv"):
        filename += ".csv"

    scrape_shopee(keyword, pages, filename)

root = tk.Tk()
root.title("Shopee Scraper GUI")
root.geometry("400x250")

tk.Label(root, text="🔍 Kata Kunci Pencarian:").pack(pady=(10, 0))
keyword_entry = tk.Entry(root, width=40)
keyword_entry.pack()

tk.Label(root, text="📄 Jumlah Halaman:").pack(pady=(10, 0))
pages_entry = tk.Entry(root, width=10)
pages_entry.pack()

tk.Label(root, text="💾 Nama File Output:").pack(pady=(10, 0))
filename_entry = tk.Entry(root, width=30)
filename_entry.pack()

ttk.Button(root, text="🚀 Mulai Scraping", command=start_scraping).pack(pady=20)

root.mainloop()


🔐 Silakan login ke Shopee.
✅ Login selesai.
✅ CAPTCHA selesai.

📄 Scraping halaman 1...
🔎 60 produk ditemukan

📄 Scraping halaman 2...
🔎 60 produk ditemukan

📄 Scraping halaman 3...
🔎 60 produk ditemukan

📄 Scraping halaman 4...
🔎 60 produk ditemukan

📄 Scraping halaman 5...
🔎 60 produk ditemukan

✅ 300 produk disimpan ke 'boardgame.csv'
